In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

In [ ]:
# Load your dataset
data_path = '/content/drive/MyDrive/bio_summary_final.csv'  # Update with your dataset path
df = pd.read_csv(data_path)

In [ ]:
# Convert the DataFrame to a Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
# Tokenization function with padding
def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["longtext"]]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)  # Adding padding

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, padding="max_length", truncation=True)  # Adding padding

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
print(df.columns)

Index(['longtext', 'summary', 'Unnamed: 2'], dtype='object')


In [ ]:
# Apply the preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
# Load the model
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./flan_t5_finetuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Set to a low batch size to fit in Colab's memory
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Optional: use a separate validation set if available
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.480046
2,No log,2.432676
3,No log,2.417479


TrainOutput(global_step=375, training_loss=2.7649137369791665, metrics={'train_runtime': 433.5616, 'train_samples_per_second': 3.446, 'train_steps_per_second': 0.865, 'total_flos': 1563985792991232.0, 'train_loss': 2.7649137369791665, 'epoch': 3.0})

In [ ]:
model.save_pretrained('/content/drive/MyDrive/flan_t5_finetuned_model')
tokenizer.save_pretrained('/content/drive/MyDrive/flan_t5_finetuned_model')

('/content/drive/MyDrive/flan_t5_finetuned_model/tokenizer_config.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/special_tokens_map.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/spiece.model',
 '/content/drive/MyDrive/flan_t5_finetuned_model/added_tokens.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model/tokenizer.json')

In [ ]:
# Define the path in Google Drive (or wherever you want to save it)
model_save_path = '/content/drive/MyDrive/flan_t5_finetuned_model_2nd'  # Update with your desired path

In [ ]:
# Save the model
trainer.model.save_pretrained(model_save_path)

In [ ]:
# Save the tokenizer
tokenizer.save_pretrained(model_save_path)

('/content/drive/MyDrive/flan_t5_finetuned_model_2nd/tokenizer_config.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/special_tokens_map.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/spiece.model',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/added_tokens.json',
 '/content/drive/MyDrive/flan_t5_finetuned_model_2nd/tokenizer.json')

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e7ffb21a6adadf2b08e81dc588b78ebacecfe317fce93d3f8cabc32837c88e8b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

# Load the ROUGE metric
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Strip white spaces
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}  # Convert to percentages

    return result


In [ ]:
import rouge_score
print("ROUGE Score Library Imported Successfully")

ROUGE Score Library Imported Successfully


In [ ]:
!pip show rouge_score

Name: rouge_score
Version: 0.1.2
Summary: Pure python implementation of ROUGE-1.5.5.
Home-page: https://github.com/google-research/google-research/tree/master/rouge
Author: Google LLC
Author-email: rouge-opensource@google.com
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, nltk, numpy, six
Required-by: 


In [ ]:
from rouge_score import rouge_scorer

# Initialize a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Test the scorer with example text
text1 = (
    "Blood is a connective tissue. It provides one of the means of communication between the cells of different parts of the body and the external environment, "
    "e.g. it carries oxygen from the lungs to the tissues and carbon dioxide from the tissues to the lungs for excretion. "
    "Blood makes up about 7% of body weight (about 5.6 litres in a 70 kg man). Blood makes up about 7% of body weight (about 5.6 litres in a 70 kg man)."
)

text2 = """
Blood is a vital connective tissue that plays a crucial role in maintaining homeostasis and facilitating communication within the body. Comprising approximately 7% of body weight, blood consists of plasma (55%) and various types of cells (45%). Plasma is a straw-colored fluid composed of 90-92% water and contains essential components such as plasma proteins, nutrients, hormones, gases, and waste products. The main plasma proteins include albumins, which help maintain osmotic pressure, and globulins, which function in immune response by neutralizing foreign invaders. Blood is responsible for transporting oxygen from the lungs to body tissues and carrying carbon dioxide from tissues back to the lungs for excretion. It also delivers nutrients absorbed from the digestive tract, hormones from endocrine glands, and waste products like urea to the kidneys for elimination. Additionally, blood's pH is slightly alkaline, ranging from 7.35 to 7.45, which is essential for optimal cellular function. The inorganic salts in blood are crucial for processes such as muscle contraction and nerve transmission. Overall, blood plays an indispensable role in sustaining life by facilitating the exchange of substances necessary for cellular health.
"""

# Calculate ROUGE scores
scores = scorer.score(text1, text2)

print(scores)


{'rouge1': Score(precision=0.2620320855614973, recall=0.5975609756097561, fmeasure=0.3643122676579925), 'rouge2': Score(precision=0.10752688172043011, recall=0.24691358024691357, fmeasure=0.14981273408239698), 'rougeL': Score(precision=0.1711229946524064, recall=0.3902439024390244, fmeasure=0.23791821561338286)}
